#Task

##The task is to build a Support Vector Machine for classification. You will make
use of a library routine to solve the optimization problem which emerges in the
dual formulation of the support vector machine. You need to write code for
structuring the data so that the library routine can find the maximal-margin
solution, and code for transforming this solution into a classifier which can
process new data.

import numpy as np
import scipy as sc
import matplotlib as mtp
